In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
import joblib
import cv2
from sklearn.preprocessing import StandardScaler


In [3]:
def data_loader(sampling, image_num, distance, features):
    """_summary_

    Args:
        image_num (int): image number to load the corresponding csv file
        distance (float): filter out distance
        features (list): list of features 
    """
    data = pd.read_csv(f'../data/processed/{sampling}/max/STD/Image_{image_num}_max_STD.csv')
    entropy = pd.read_csv(f'../data/processed/{sampling}/max/Entropy/Image_{image_num}_max_Entropy.csv')
    depth = pd.read_csv(f'../data/processed/{sampling}/max/STD/Image_{image_num}_max_STD.csv')
    
    new_df = pd.DataFrame()

    new_df['max_STD'] = data['max_STD']
    new_df['max_Entropy'] = entropy['max_Entropy']
    new_df['depth_value'] = depth['depth_value'] 
    new_df['x'] = depth['x'] 
    new_df['y'] = depth['y'] 
    
    new_df["label"] = np.where(new_df['depth_value'] <= distance, 1, 0)

    max_labels = new_df[new_df.label == 0]
    min_labels = new_df[new_df.label == 1]

    max_down = resample(max_labels,
                        replace=False,
                        n_samples=len(min_labels),     # match minority
                        random_state=42)

    df_balanced = pd.concat([max_down, min_labels])
    
    return df_balanced

In [ ]:
DIR = "out"
SAMPLING = "TopDown"
# IMAGE_NUM = 5

# TopDown (Done): 
#          3CM , 5CM, 0.009CM
#          STD, Entropy, (Std, Entropy)

# Circular : 
#          3CM , 5CM, 0.009 CM
#          STD, Entropy, (Std, Entropy)

# FEATURES = ['max_STD']
# FEATURES = ['max_Entropy']
FEATURES = ['max_STD', 'max_Entropy']

DISTANCE = 0.03

feats = "_".join([i.split("_")[-1].upper() for i in FEATURES])
EXP_NAME = feats+"_"+str(DISTANCE)+"_CM"

try:
    if not os.path.isdir(os.path.join("../", DIR, SAMPLING, EXP_NAME)):
        os.makedirs(os.path.join("../",DIR, SAMPLING, EXP_NAME))
except:
    print("Folder Exist")

accuracy_curve = []

for img_num in range(1, 101):
    # FOLDER_NAME = "IMAGE_NUM_"+str(img_num)
    if img_num == 100:
        DISTANCE = 0.029
    data = data_loader(SAMPLING, img_num, DISTANCE, FEATURES)
    X_train, X_test, y_train, y_test = train_test_split(data, data, test_size=0.2, random_state=42)

    data_len = len(X_test)

    image = np.zeros((1024, 1024))
    for i in range(data_len):
        X = int(X_test.iloc[i]['x'])
        Y = int(X_test.iloc[i]['y'])
        
        image[X, Y] = int(X_test.iloc[i]['label'])
        

    # try:
    #     if not os.path.isdir(os.path.join("../", DIR, SAMPLING, EXP_NAME, FOLDER_NAME)):
    #         os.makedirs(os.path.join("../",DIR, SAMPLING, EXP_NAME, FOLDER_NAME))
    # except:
    #     print("Folder Exist")
    
    cv2.imwrite(os.path.join("../", DIR, SAMPLING, EXP_NAME, f"{img_num}.png"), image * 255)

  


# To Test the model

In [ ]:
DIR = "out"
MODEL_DIR = "logs"
SAMPLING = "TopDown"
# IMAGE_NUM = 5

# TopDown (Done): 
#          3CM , 5CM, 0.009CM
#          STD, Entropy, (Std, Entropy)

# Circular : 
#          3CM , 5CM, 0.009 CM
#          STD, Entropy, (Std, Entropy)

# FEATURES = ['max_STD']
# FEATURES = ['max_Entropy']
FEATURES = ['max_STD', 'max_Entropy']

DISTANCE = 0.03

feats = "_".join([i.split("_")[-1].upper() for i in FEATURES])
EXP_NAME = feats+"_"+str(DISTANCE)+"_CM"

try:
    if not os.path.isdir(os.path.join("../", DIR, SAMPLING, EXP_NAME)):
        os.makedirs(os.path.join("../",DIR, SAMPLING, EXP_NAME))
except:
    print("Folder Exist")

accuracy_curve = []

for img_num in range(1, 101):
    FOLDER_NAME = "IMAGE_NUM_"+str(img_num)
    if img_num == 100:
        DISTANCE = 0.029
    data = data_loader(SAMPLING, img_num, DISTANCE, FEATURES)
    X_train, X_test, y_train, y_test = train_test_split(data, data, test_size=0.2, random_state=42)
    model = joblib.load(os.path.join("../", MODEL_DIR, SAMPLING, EXP_NAME, FOLDER_NAME, "model.pkl")) 

  
    training_feats = np.array(X_train[['max_STD', 'max_Entropy']])

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(training_feats)
    # X_test_scaled = scaler.transform(X_test)
    # print(X_test_scaled)
    # sdf


    data_len = len(X_test)
    image = np.zeros((1024, 1024))
    for i in range(data_len):
        X = int(X_test.iloc[i]['x'])
        Y = int(X_test.iloc[i]['y'])
        
        feats = np.array([[X_test.iloc[i]['max_STD'], X_test.iloc[i]['max_Entropy']]])
        image[X, Y] = int(model.predict(scaler.transform(feats)).item())
        

    # try:
    #     if not os.path.isdir(os.path.join("../", DIR, SAMPLING, EXP_NAME, FOLDER_NAME)):
    #         os.makedirs(os.path.join("../",DIR, SAMPLING, EXP_NAME, FOLDER_NAME))
    # except:
    #     print("Folder Exist")
    
    cv2.imwrite(os.path.join("../", DIR, SAMPLING, EXP_NAME, f"{img_num}.png"), image * 255)

  


In [ ]:
accuracy_avg = np.load('..\logs\TopDown\STD_ENTROPY_0.03_CM\STD_ENTROPY_0.03_CM_accuracy_curve.npy')
np.mean(accuracy_avg)

0.792016603124342